In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.model_selection import train_test_split

In [ ]:
target_path = "C:\\Users\\batuy\\Documents\\VS Code\\animal-classification-app-flask\\Data"

In [ ]:
img_size = (224,224)

In [ ]:
import os
import shutil

data_dir = target_path
train_dir = "train_data"
test_dir = "test_data"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
# Her bir kategori için veri ayırma işlemi
for category in os.listdir(data_dir):
    category_path = os.path.join(data_dir, category)
    if os.path.isdir(category_path):
        
        files = os.listdir(category_path)
        train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
        
        train_category_dir = os.path.join(train_dir, category)
        test_category_dir = os.path.join(test_dir, category)
        
        os.makedirs(train_category_dir, exist_ok=True)
        os.makedirs(test_category_dir, exist_ok=True)
        
        for file in train_files:
            shutil.copy(os.path.join(category_path, file), os.path.join(train_category_dir, file))
        
        for file in test_files:
            shutil.copy(os.path.join(category_path, file), os.path.join(test_category_dir, file))


In [ ]:
train_data_gen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_d = train_data_gen.flow_from_directory(directory="train_data", class_mode="categorical", batch_size=64, target_size=img_size)

test_d = test_data_gen.flow_from_directory(directory="train_data", class_mode="categorical", batch_size=64, target_size=img_size)

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential

In [ ]:
model = Sequential()

model.add(Convolution2D(32, (2, 2), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.summary()

In [ ]:
from keras.layers import Dropout

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
#callbacks

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
rlp = ReduceLROnPlateau(monitor='val_accuracy', factor=0.3, min_lr=0.000001, patience=2)
model_check = ModelCheckpoint(filepath="project.keras", monitor="val_loss", verbose=1 ,save_best_only=True)
early_ = EarlyStopping(monitor="val_loss",
                        min_delta=0.2,
                        patience=10,
                        verbose=1,
                        mode="auto",
                        baseline=None,
                        restore_best_weights=False,
                        start_from_epoch=0,)


callbacks = [rlp, early_, model_check]

In [ ]:
history = model.fit(train_d, validation_data=test_d, batch_size=32, epochs=25, callbacks=callbacks)

In [ ]:
from keras.models import load_model

In [ ]:
model_2 = load_model("project.keras")

In [ ]:
loss, accuracy = model_2.evaluate(test_d)
print(f"loss: {loss}, accuracy: {accuracy}")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Kayıtlı modeli yükle
model = load_model("project.keras")

# Test edilecek görüntü dosyasının yolu
img_path = "C:\\Users\\batuy\\Documents\\VS Code\\animal-classification-app-flask\\test_data\\Elephant\\Elephant_91.jpg"  # Test edilecek görüntü dosyasının yolunu belirtin

# Görüntüyü modelin giriş şekline uygun olarak yükle ve ön işleme yap
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalizasyon

# Tahmin yap
predictions = model.predict(img_array)

# Sınıf adlarını belirleyin (modelinize göre güncelleyin)
class_names = ["Buffalo", "zebra", "Elephant", "Rhino"]  # Örnek: ['cats', 'dogs', 'horses']

# Tahmin sonucunu görselleştirme
plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plt.imshow(img, cmap=plt.cm.binary)
plt.axis('off')

predicted_class = np.argmax(predictions)
confidence = np.max(predictions) * 100
class_label = class_names[predicted_class]

plt.title(f"Predicted: {class_label}\nConfidence: {confidence:.2f}%")

plt.subplot(1, 2, 2)
plt.bar(class_names, predictions.reshape(-1))
plt.xlabel('Classes')
plt.ylabel('Probability')
plt.title('Class Probabilities')
plt.ylim([0, 1])
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()